In [1]:
import os
path = "/Users/patricia/Documents/code/python-code/behavior-detection/src"
os.chdir(path)  # Muda o diretório para o nível anterior (a raiz do projeto)
print(os.getcwd())  # Verifique se agora está na raiz

/Users/patricia/Documents/code/python-code/behavior-detection/src


# Load data

In [2]:
from behavior.behavior_data_loader import BehaviorDataLoader

data_path = '../data/new_logs_labels.csv'

data = BehaviorDataLoader.load_data(data_path, delimiter=';')
data.head(5)

,id_log,aluno,grupo,num_dia,num_log,log_type,ultimo_passo_correto,verificado_com_mouse,verificado_com_teclado,idle_time_acumulado,...,comportamento_off_task,comportamento_on_system,comportamento_indefinido,ultimo_comportamento,ultimo_comportamento_on_task,ultimo_comportamento_on_task_conversation,ultimo_comportamento_on_task_out,ultimo_comportamento_off_task,ultimo_comportamento_on_system,ultimo_comportamento_indefinido
0,8224,1,2,1,1,step_verification,0,0,1,0,...,0,0,0,?,0,0,0,0,0,1
1,527786,1,2,1,2,user_idle,1,0,0,2,...,0,0,0,ON TASK,1,0,0,0,0,0
2,527787,1,2,1,3,user_idle,0,0,0,4,...,0,0,0,ON TASK,1,0,0,0,0,0
3,527788,1,2,1,4,user_idle,0,0,0,6,...,0,0,0,ON TASK,1,0,0,0,0,0
4,527789,1,2,1,5,user_idle,0,0,0,8,...,0,0,0,ON TASK,1,0,0,0,0,0


In [3]:
from core.preprocessors.data_cleaner import DataCleaner

print("Valores da coluna 'comportamento' antes da remoção:", data['comportamento'].value_counts())

# Remove instances where 'comportamento' is '?'
data = DataCleaner.remove_instances_with_value(data, 'comportamento', '?')

print("\nValores da coluna 'comportamento' depois da remoção:", data['comportamento'].value_counts())

Valores da coluna 'comportamento' antes da remoção: comportamento
ON TASK                 3159
ON SYSTEM                907
OFF TASK                 629
ON TASK CONVERSATION     414
ON TASK OUT              380
?                         36
Name: count, dtype: int64

Valores da coluna 'comportamento' depois da remoção: comportamento
ON TASK                 3159
ON SYSTEM                907
OFF TASK                 629
ON TASK CONVERSATION     414
ON TASK OUT              380
Name: count, dtype: int64


In [4]:
data.head(5)

,id_log,aluno,grupo,num_dia,num_log,log_type,ultimo_passo_correto,verificado_com_mouse,verificado_com_teclado,idle_time_acumulado,...,comportamento_off_task,comportamento_on_system,comportamento_indefinido,ultimo_comportamento,ultimo_comportamento_on_task,ultimo_comportamento_on_task_conversation,ultimo_comportamento_on_task_out,ultimo_comportamento_off_task,ultimo_comportamento_on_system,ultimo_comportamento_indefinido
0,8224,1,2,1,1,step_verification,0,0,1,0,...,0,0,0,?,0,0,0,0,0,1
1,527786,1,2,1,2,user_idle,1,0,0,2,...,0,0,0,ON TASK,1,0,0,0,0,0
2,527787,1,2,1,3,user_idle,0,0,0,4,...,0,0,0,ON TASK,1,0,0,0,0,0
3,527788,1,2,1,4,user_idle,0,0,0,6,...,0,0,0,ON TASK,1,0,0,0,0,0
4,527789,1,2,1,5,user_idle,0,0,0,8,...,0,0,0,ON TASK,1,0,0,0,0,0


In [5]:
## Select a subset of the data only for testing purposes

# Selecionar um subconjunto dos dados
# print("Tamanho do dataframe antes:", data.shape)
data = data.sample(n=500, random_state=42)
data.reset_index(drop=True, inplace=True)
# print("Tamanho do dataframe após:", data.shape)

# Pre-processing

## Remove unnecessary columns

In [6]:
# Removing columns related to IDs, emotions, personality and behaviors, because 
# we want to classify behaviors only by the students' interactions with the system
columns_to_remove_ids = ['id_log', 'grupo', 'num_dia', 'num_log']
columns_to_remove_emotions = [
    'estado_afetivo', 'estado_engajamento_concentrado', 
    'estado_confusao', 'estado_frustracao', 'estado_tedio', 'estado_indefinido', 
    'ultimo_estado_afetivo', 'ultimo_engajamento_concentrado', 'ultimo_confusao', 
    'ultimo_frustracao', 'ultimo_tedio', 'ultimo_estado_indefinido'
]
columns_to_remove_personality = [
    'traco_amabilidade_fator', 'traco_extrovercao_fator', 'traco_conscienciosidade_fator', 
    'traco_abertura_fator', 'traco_neuroticismo_fator', 'traco_amabilidade_cat', 
    'traco_extrovercao_cat', 'traco_conscienciosidade_cat', 'traco_abertura_cat', 
    'traco_neuroticismo_cat']

columns_to_remove_behaviors = [
    'comportamento_on_task', 'comportamento_on_task_conversation', 'comportamento_on_task_out',
    'comportamento_off_task', 'comportamento_on_system', 'comportamento_indefinido',
    'ultimo_comportamento', 'ultimo_comportamento_on_task', 'ultimo_comportamento_on_task_conversation',
    'ultimo_comportamento_on_task_out', 'ultimo_comportamento_off_task', 'ultimo_comportamento_on_system',
    'ultimo_comportamento_indefinido'
]

columns_to_remove = columns_to_remove_ids + \
        columns_to_remove_emotions + \
        columns_to_remove_personality + \
        columns_to_remove_behaviors

cleaned_data = DataCleaner.remove_columns(data, columns_to_remove)


In [7]:
cleaned_data.head(5)

,aluno,log_type,ultimo_passo_correto,verificado_com_mouse,verificado_com_teclado,idle_time_acumulado,num_click_acumulado,num_click_passo,num_click_eq,type_step_verification,...,misc_OI_Dv_Plus_Sb_total,misc_EqSec_Distrib_MtTerm_total,misc_OI_Mt_Minus_Mt_Plus_total,misc_OI_Mt_Minus_Mt_Minus_total,misc_OI_Dv_Plus_Ad_total,misc_EqPrim_Mt_Inc_total,misc_EqPrim_Dv_Inc_total,misc_OI_Dv_Minus_Dv_Minus_total,misc_EqSec_OpFrac_MMC_MtNumerador_total,comportamento
0,26,user_idle,0,0,0,16,0,0,4,0,...,0,9,0,0,0,1,0,0,0,OFF TASK
1,1,user_idle,0,0,0,2,0,3,3,0,...,0,0,0,0,0,0,0,0,0,ON TASK
2,1,click,0,0,0,0,2,2,2,0,...,0,0,0,0,0,0,0,0,0,ON TASK
3,21,user_idle,0,0,0,6,0,1,1,0,...,0,1,0,0,0,0,0,0,0,ON TASK
4,27,click,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,ON SYSTEM


In [8]:
# Preenche valores ausentes no DataFrame X com a string 'missing'.

cleaned_data = cleaned_data.fillna('missing')

## Split data by student level into training and test datasets

In [9]:
from core.preprocessors.data_splitter import DataSplitter

train_data, test_data = DataSplitter.split_by_student_level(cleaned_data, test_size=0.2, column_name='aluno')

In [10]:
# removing the 'aluno' column from the data after splitting into train and test sets

# Remover 'aluno' do conjunto de treinamento
cleaned_data = DataCleaner.remove_columns(train_data, ['aluno'])

# Remover 'aluno' do conjunto de teste
cleaned_data = DataCleaner.remove_columns(test_data, ['aluno'])

## Split data into Features (X) and Target (y)

In [11]:
from core.preprocessors.data_splitter import DataSplitter

# Conjunto de treinamento
X_train, y_train = DataSplitter.split_into_x_y(train_data, 'comportamento')

# Conjunto de teste
X_test, y_test = DataSplitter.split_into_x_y(test_data, 'comportamento')

In [12]:
print("Primeiras 5 instâncias de y_train:")
print(y_train[:5])

print("\nPrimeiras 5 instâncias de y_test:")
print(y_test[:5])

Primeiras 5 instâncias de y_train:
0     OFF TASK
1      ON TASK
2      ON TASK
3      ON TASK
4    ON SYSTEM
Name: comportamento, dtype: object

Primeiras 5 instâncias de y_test:
11                 ON TASK
12             ON TASK OUT
19                 ON TASK
22               ON SYSTEM
39    ON TASK CONVERSATION
Name: comportamento, dtype: object


## Encoding variables

### Encoding true labels (y)

In [13]:
import importlib
from core.preprocessors import column_selector, data_encoder
from behavior import behavior_data_encoder

# Recarregar o módulo para garantir que as alterações sejam aplicadas
importlib.reload(column_selector)
importlib.reload(data_encoder)
importlib.reload(behavior_data_encoder)

<module 'behavior.behavior_data_encoder' from '/Users/patricia/Documents/code/python-code/behavior-detection/src/behavior/behavior_data_encoder.py'>

In [14]:
# Encoding y_train and y_test
from behavior.behavior_data_encoder import BehaviorDataEncoder

# Codificar y_train
y_train = BehaviorDataEncoder.encode_y(y_train)

# Codificar y_test
y_test = BehaviorDataEncoder.encode_y(y_test)

### Encoding features (X)

In [15]:
# Pré-processar X_train
X_encoder = BehaviorDataEncoder(num_classes=5)
X_encoder.fit(X_train)

X_train = X_encoder.transform(X_train)

# Pré-processar X_test usando o mesmo preprocessor
X_test = X_encoder.transform(X_test)

# Balanceamento dos dados

In [16]:
from core.preprocessors.data_balancer import DataBalancer

data_balancer = DataBalancer()
X_train, y_train = data_balancer.apply_smote(X_train, y_train)

In [17]:
from collections import Counter

print(f"Resampled dataset shape: {Counter(y_train)}")

Resampled dataset shape: Counter({0: 244, 2: 244, 1: 244, 4: 244, 3: 244})


# Treinamento dos Modelos

## Definindo parametros

In [18]:
# Definir quais modelos e seletores utilizar
selected_models = ['Decision Tree']
selected_selectors = ['pca']

cv = 2  # Number of folds in the cross-validation
n_iter = 100
n_jobs = 4  # Number of processors to be used in the execution: -1 to use all processors

# Choose a scoring metric
scoring_metric = 'roc_auc_ovr'  # Possible values: 'f1_macro', 'balanced_accuracy', 'roc_auc_ovr', etc.

## Usando Otimização Bayesiana (BayesSearchCV)

In [19]:
from core.training.skopt_bayesian_optimization_training import SkoptBayesianOptimizationTraining

training = SkoptBayesianOptimizationTraining()

#### Executar o treinamento
trained_models = training.train_model(
    X_train=X_train,
    y_train=y_train,
    selected_models=selected_models,
    selected_selectors=selected_selectors,
    n_iter=n_iter,
    cv=cv,
    scoring=scoring_metric,
    n_jobs=n_jobs
)

#### Exemplo de acesso aos modelos treinados
for model_key, model_info in trained_models.items():
    print(f"Modelo: {model_key}")
    print(f"Melhores Hiperparâmetros: {model_info['hyperparameters']}")
    print(f"Resultado CV: {model_info['cv_result']}\n") 

INFO:root:Training and evaluating Decision Tree with Bayesian Optimization and pca
INFO:root:Search space for Decision Tree with selector pca: {'classifier__max_depth': Categorical(categories=(None, 3, 5, 10, 20, 30), prior=None), 'classifier__min_samples_split': Integer(low=2, high=20, prior='uniform', transform='identity'), 'classifier__min_samples_leaf': Integer(low=1, high=10, prior='uniform', transform='identity'), 'feature_selection__n_components': [1, 5, 10, 20, 30, 40, 50]}


Training and evaluating Decision Tree with Bayesian Optimization and pca
Fitting 2 folds for each of 1 candidates, totalling 2 fits


INFO:root:Iteration 1: tested parameters: [5, 8, 19, 10], score: 0.8369599905939265


[CV 1/2] END classifier__max_depth=5, classifier__min_samples_leaf=8, classifier__min_samples_split=19, feature_selection__n_components=10;, score=0.837 total time=   0.0s
[CV 2/2] END classifier__max_depth=5, classifier__min_samples_leaf=8, classifier__min_samples_split=19, feature_selection__n_components=10;, score=0.837 total time=   0.0s
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 2/2] END classifier__max_depth=30, classifier__min_samples_leaf=9, classifier__min_samples_split=7, feature_selection__n_components=50;, score=0.899 total time=   0.1s


INFO:root:Iteration 2: tested parameters: [30, 9, 7, 50], score: 0.883982800322494
INFO:root:Iteration 3: tested parameters: [5, 9, 4, 20], score: 0.8504493079817254


...

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END classifier__max_depth=30, classifier__min_samples_leaf=10, classifier__min_samples_split=11, feature_selection__n_components=10;, score=0.875 total time=   0.0s
[CV 2/2] END classifier__max_depth=30, classifier__min_samples_leaf=10, classifier__min_samples_split=11, feature_selection__n_components=10;, score=0.898 total time=   0.0s


INFO:root:Iteration 99: tested parameters: [30, 10, 11, 10], score: 0.8863603533996238


Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END classifier__max_depth=None, classifier__min_samples_leaf=10, classifier__min_samples_split=2, feature_selection__n_components=10;, score=0.872 total time=   0.1s
[CV 2/2] END classifier__max_depth=None, classifier__min_samples_leaf=10, classifier__min_samples_split=2, feature_selection__n_components=10;, score=0.898 total time=   0.1s


INFO:root:Iteration 100: tested parameters: [None, 10, 2, 10], score: 0.8852593388873958
INFO:root:Bayesian optimization results: Best result: 0.9003292125772642, Average cross-validation result: 0.9003292125772642
INFO:root:Bayesian Optimization Best Result for Decision Tree with pca: 0.9003292125772642


Bayesian Optimization Best Result for Decision Tree with pca: 0.9003292125772642
Modelo: Decision Tree_pca
Melhores Hiperparâmetros: {'memory': None, 'steps': [('feature_selection', PCA(n_components=20)), ('classifier', DecisionTreeClassifier(max_depth=20, min_samples_split=18))], 'verbose': False, 'feature_selection': PCA(n_components=20), 'classifier': DecisionTreeClassifier(max_depth=20, min_samples_split=18), 'feature_selection__copy': True, 'feature_selection__iterated_power': 'auto', 'feature_selection__n_components': 20, 'feature_selection__n_oversamples': 10, 'feature_selection__power_iteration_normalizer': 'auto', 'feature_selection__random_state': None, 'feature_selection__svd_solver': 'auto', 'feature_selection__tol': 0.0, 'feature_selection__whiten': False, 'classifier__ccp_alpha': 0.0, 'classifier__class_weight': None, 'classifier__criterion': 'gini', 'classifier__max_depth': 20, 'classifier__max_features': None, 'classifier__max_leaf_nodes': None, 'classifier__min_impurit

## Usando Otimização Bayesiana (Optuna)

In [20]:
# Importação da nova classe OptunaBayesianOptimizationTraining
from core.training.optuna_bayesian_optimization_training import OptunaBayesianOptimizationTraining

# Configurações de validação cruzada e otimização
cv = 2  # Número de folds na validação cruzada
n_iter = 100  # Número de iterações/trials para otimização
n_jobs = 4  # Número de processadores a serem utilizados na execução: -1 para usar todos os processadores

# Escolha da métrica de avaliação
scoring_metric = 'roc_auc_ovr'  # Possíveis valores: 'f1_macro', 'balanced_accuracy', 'roc_auc_ovr', etc.

# Instanciação da classe de treinamento com Otimização Bayesiana via Optuna
training = OptunaBayesianOptimizationTraining()

# Definir quais modelos e seletores utilizar
selected_models = ['Decision Tree']  # Exemplos: ['Random Forest', 'SVM', 'XGBoost']
selected_selectors = ['pca']  # Exemplos: ['pca', 'selectkbest']

# Executar o treinamento
trained_models = training.train_model(
    X_train=X_train,
    y_train=y_train,
    selected_models=selected_models,
    selected_selectors=selected_selectors,
    n_iter=n_iter,  # Será mapeado para n_trials na classe OptunaBayesianOptimizationTraining
    cv=cv,
    scoring=scoring_metric,
    n_jobs=n_jobs
)

# Exemplo de acesso aos modelos treinados
for model_key, model_info in trained_models.items():
    print(f"Modelo: {model_key}")
    print(f"Melhores Hiperparâmetros: {model_info['hyperparameters']}")
    print(f"Resultado CV: {model_info['cv_result']}\n")


2024-10-16 16:13:41,230 | optuna_bayesian_optimization | INFO | Training and evaluating Decision Tree with Optuna Bayesian Optimization and pca
INFO:optuna_bayesian_optimization:Training and evaluating Decision Tree with Optuna Bayesian Optimization and pca
[I 2024-10-16 16:13:41,232] A new study created in memory with name: no-name-dd04694e-e8b3-4ee6-b91b-9aef55987aea


Training and evaluating Decision Tree with Optuna Bayesian Optimization and pca


DEBUG:optuna_bayesian_optimization:Trial 0: Hyperparameters: {'classifier__max_depth': None, 'classifier__min_samples_split': 15, 'classifier__min_samples_leaf': 3}, Score: 0.8803555831765654
[I 2024-10-16 16:13:41,393] Trial 0 finished with value: 0.8803555831765654 and parameters: {'classifier__max_depth': None, 'classifier__min_samples_split': 15, 'classifier__min_samples_leaf': 3}. Best is trial 0 with value: 0.8803555831765654.
DEBUG:optuna_bayesian_optimization:Trial 1: Hyperparameters: {'classifier__max_depth': 10, 'classifier__min_samples_split': 2, 'classifier__min_samples_leaf': 4}, Score: 0.879461166353131
[I 2024-10-16 16:13:41,486] Trial 1 finished with value: 0.879461166353131 and parameters: {'classifier__max_depth': 10, 'classifier__min_samples_split': 2, 'classifier__min_samples_leaf': 4}. Best is trial 0 with value: 0.8803555831765654.
DEBUG:optuna_bayesian_optimization:Trial 2: Hyperparameters: {'classifier__max_depth': 30, 'classifier__min_samples_split': 8, 'classi

---Optuna Bayesian Optimization---
Total time: 14.73 seconds
Number of trials: 100
Best trial index: 70
Best score: 0.8864376175759205
Best hyperparameters: {'classifier__max_depth': None, 'classifier__min_samples_split': 15, 'classifier__min_samples_leaf': 2}


2024-10-16 16:13:56,417 | optuna_bayesian_optimization | INFO | Optuna Optimization Best Result for Decision Tree with pca: 0.8864376175759205
INFO:optuna_bayesian_optimization:Optuna Optimization Best Result for Decision Tree with pca: 0.8864376175759205


Optuna Optimization Best Result for Decision Tree with pca: 0.8864376175759205
Modelo: Decision Tree_pca
Melhores Hiperparâmetros: {'classifier__max_depth': None, 'classifier__min_samples_split': 15, 'classifier__min_samples_leaf': 2}
Resultado CV: 0.8864376175759205



# Avaliação dos Modelos

In [21]:
from core.evaluation.evaluation import Evaluation  

feature_names = X_train.columns  # Assumindo que os nomes das características são as colunas
class_metrics_results, avg_metrics_results = Evaluation.evaluate_all_models(trained_models, X_train, y_train, X_test, y_test, feature_names)

/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


# Geração dos Relatórios

In [22]:
from core.logging.report_formatter import ReportFormatter
from core.logging.file_manager import FileManager

directory = "../output/"

# Gerar relatório textual a partir dos resultados de avaliação
text_report = ReportFormatter.generate_text_report(class_metrics_results, avg_metrics_results)

# Imprimir ou salvar o relatório
FileManager.save_file_with_timestamp(text_report, "bayesian_optimization_report.txt", directory)

# Gerar DataFrame detalhado dos relatórios por classe
class_report_df = ReportFormatter.generate_class_report_dataframe(class_metrics_results)

# Gerar DataFrame resumido dos relatórios de métricas médias
avg_metrics_report_df = ReportFormatter.generate_avg_metrics_report_dataframe(avg_metrics_results)

# Salvar os DataFrames como arquivos CSV, se necessário
FileManager.save_csv_file_with_timestamp(class_report_df, "class_report.csv", directory)
FileManager.save_csv_file_with_timestamp(avg_metrics_report_df, "avg_metrics_report.csv", directory)


'../output/avg_metrics_report_20241016_1613.csv'

# Salvando os modelos em arquivos para recuperação

In [23]:
from core.logging.model_manager import ModelManager

# Caminhos
model_dir = "../models/"

# Salvar todos os modelos
saved_models = ModelManager.save_all_models(trained_models, model_dir)
print("Modelos salvos:", saved_models)

Model 'Decision Tree_pca' saved at: ../models/model_Decision Tree_pca_20241016_1613.pkl
Modelos salvos: ['../models/model_Decision Tree_pca_20241016_1613.pkl']
